In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [30]:
# 1. Load dataset heart.csv
df = pd.read_csv("../Data/heart.csv")

# Asumsi kolom terakhir adalah target
X = df.drop("target", axis=1).values
y = df["target"].values

In [31]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [33]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)


In [34]:
def build_model(dropout1=0.0, dropout2=0.0):
    layers = [
        nn.Linear(X_train.shape[1], 32),
        nn.ReLU(),
    ]

    if dropout1 > 0:
        layers.append(nn.Dropout(dropout1))

    layers.extend([
        nn.Linear(32, 16),
        nn.ReLU(),
    ])

    if dropout2 > 0:
        layers.append(nn.Dropout(dropout2))
        
    layers.extend([
        nn.Linear(16, 2)
    ])
    return nn.Sequential(*layers)

In [35]:
def train_with_dropout(dropout1=0.5, dropout2=0.3, epochs=50):
    print(f"\n=== Training with Dropout={dropout1} ===")
    model = build_model(dropout1=dropout1, dropout2=dropout2)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            with torch.no_grad():
                preds = model(X_test)
                _, predicted = torch.max(preds, 1)
                acc = (predicted == y_test).sum().item() / len(y_test)
            print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc:.4f}")

In [36]:
def train_with_l2(l2_lambda=0.01, epochs=50):
    print(f"\n=== Training with L2 (lambda={l2_lambda}) ===")
    model = build_model(dropout1=0.0, dropout2=0.0)
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=l2_lambda)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            with torch.no_grad():
                preds = model(X_test)
                _, predicted = torch.max(preds, 1)
                acc = (predicted == y_test).sum().item() / len(y_test)
            print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc:.4f}")

In [37]:
def train_with_l1(l1_lambda=0.001, epochs=50):
    print(f"\n=== Training with L1 (lambda={l1_lambda}) ===")
    model = build_model(dropout1=0.0, dropout2=0.0)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Tambahkan L1 penalty
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = loss + l1_lambda * l1_norm

        loss.backward()
        optimizer.step()

        if (epoch+1) % 10 == 0:
            with torch.no_grad():
                preds = model(X_test)
                _, predicted = torch.max(preds, 1)
                acc = (predicted == y_test).sum().item() / len(y_test)
            print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc:.4f}")

In [38]:
# 5. Eksperimen
train_with_dropout(dropout1=0.5, dropout2=0.3, epochs=50)
train_with_l2(l2_lambda=0.01, epochs=50)
train_with_l1(l1_lambda=0.001, epochs=50)


=== Training with Dropout=0.5 ===
Epoch 10: Loss=0.5172, Acc=0.7951
Epoch 20: Loss=0.3918, Acc=0.8146
Epoch 30: Loss=0.3547, Acc=0.8439
Epoch 40: Loss=0.3069, Acc=0.8341
Epoch 50: Loss=0.3072, Acc=0.8488

=== Training with L2 (lambda=0.01) ===
Epoch 10: Loss=0.3772, Acc=0.8195
Epoch 20: Loss=0.3143, Acc=0.8488
Epoch 30: Loss=0.2758, Acc=0.8683
Epoch 40: Loss=0.2243, Acc=0.8878
Epoch 50: Loss=0.1719, Acc=0.9268

=== Training with L1 (lambda=0.001) ===
Epoch 10: Loss=0.4979, Acc=0.8049
Epoch 20: Loss=0.4271, Acc=0.8488
Epoch 30: Loss=0.3772, Acc=0.8683
Epoch 40: Loss=0.3389, Acc=0.8878
Epoch 50: Loss=0.3009, Acc=0.9024
